## Behavioral Cloning Project
Created: Florin Gheorghe  
Date: 18/03/2018

In [ ]:
# Import modules
import csv
import cv2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import random
from PIL import Image

In [ ]:
# Placeholder to load the driving data
images = []
steering = []

In [ ]:
# correction for steering
corr = 0.15

In [ ]:
lines = []
with open('.\\driving_data\\driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
for line in lines:
    # Left camera images
    source_path = line[1]
    filename = source_path.split('\\')[-1]
    image = mpimg.imread('.\\driving_data\\IMG\\'+filename)
    images.append(image)
    angle = float(line[3])
    steering.append(angle+corr)
    
    # Right camera images
    source_path = line[2]
    filename = source_path.split('\\')[-1]
    image = mpimg.imread('.\\driving_data\\IMG\\'+filename)
    images.append(image)
    angle = float(line[3])
    steering.append(angle-corr)

In [ ]:
# Augmenting data by flipping and reversing angle
x, y = [] , []
for image, angle in zip(images, steering):
    x.append(image)
    y.append(angle)
    x.append(cv2.flip(image,1))
    y.append(angle*(-1.0))

In [ ]:
# Visualize one random image
index = random.randint(0, len(x))
image = x[index].squeeze()
plt.figure(figsize=(10,6))

plt.imshow(image)
print(y[index])

In [ ]:
import keras as ks
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Activation
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from sklearn.utils import shuffle

In [ ]:
# Initiating the model
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70, 25), (0, 0))))

model.add(Conv2D(24, (5,5), activation='relu'))
model.add(Conv2D(36, (5, 5), activation='relu'))
model.add(Conv2D(48, (3,3), activation='relu'))

model.add(Flatten())
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

In [ ]:
X_train, y_train = shuffle(np.array(x), np.array(y), random_state=3)

In [ ]:
adam = ks.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='mse', optimizer=adam)
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=3)

In [ ]:
model.save('./model.h5')